In [0]:
#This is ADLS Gen-2 accountname and access key details
storageaccount="cookbookadlsg2storage"
acct_info=f"fs.azure.account.key.{storageaccount}.dfs.core.windows.net"
accesskey="YYax32MlhsocFCKQokLi85tHEQNoK/32aVOfpfyyv5pHM48iQvYIXU4erZHk1JIw349DmLKHt3ln+AStRaYXiw=="
print(acct_info)

fs.azure.account.key.cookbookadlsg2storage.dfs.core.windows.net


In [0]:
#setting account credentials in notebook session configs
spark.conf.set(
acct_info,
accesskey)

In [0]:
cust_path = "abfss://rawdata@cookbookadlsg2storage.dfs.core.windows.net/customers/parquetFiles"

In [0]:
df_cust = (spark.read.format("parquet").load(cust_path))
df_cust.write.format("delta").mode("overwrite").save("abfss://rawdata@cookbookadlsg2storage.dfs.core.windows.net/customers/delta")

In [0]:
%sql
--creating Delta table
DROP TABLE IF EXISTS Customer;
CREATE TABLE Customer
USING delta
location "abfss://rawdata@cookbookadlsg2storage.dfs.core.windows.net/customers/delta"


In [0]:
%sql
describe formatted Customer


col_name,data_type,comment
C_CUSTKEY,int,null
C_NAME,string,null
C_ADDRESS,string,null
C_NATIONKEY,int,null
C_PHONE,string,null
C_ACCTBAL,"decimal(13,2)",null
C_MKTSEGMENT,string,null
C_COMMENT,string,null
,,
# Detailed Table Information,,


In [0]:
ord_path = "abfss://rawdata@cookbookadlsg2storage.dfs.core.windows.net/orders/parquetFiles"
df_ord = (spark.read.format("parquet").load(ord_path))
df_ord.write.format("delta").mode("overwrite").save("abfss://rawdata@cookbookadlsg2storage.dfs.core.windows.net/orders/delta")

In [0]:
%sql
DROP TABLE IF EXISTS Orders;
CREATE TABLE Orders
USING delta
location "abfss://rawdata@cookbookadlsg2storage.dfs.core.windows.net/orders/delta"

In [0]:
%sql
describe formatted Orders


col_name,data_type,comment
O_ORDERKEY,int,null
O_CUSTKEY,int,null
O_ORDERSTATUS,string,null
O_TOTALPRICE,"decimal(13,2)",null
O_ORDERDATE,date,null
O_ORDERPRIORITY,string,null
O_CLERK,string,null
O_SHIPPRIORITY,int,null
O_COMMENT,string,null
,,


In [0]:
%sql
-- Getting total customer based on priority and total account balance by joining the delta tables
SELECT o.O_ORDERPRIORITY,count(o.O_CUSTKEY) As TotalCustomer,Sum(c.C_ACCTBAL) As CustAcctBal
FROM Orders o INNER JOIN Customer c on o.O_CUSTKEY=c.C_CUSTKEY
WHERE o.O_ORDERDATE>'1997-12-31'
GROUP BY o.O_ORDERPRIORITY
ORDER BY TotalCustomer DESC;


O_ORDERPRIORITY,TotalCustomer,CustAcctBal
5-LOW,23411,104725205.21
3-MEDIUM,23227,104342472.34
1-URGENT,23116,104059187.35
2-HIGH,23044,103845558.06
4-NOT SPECIFIED,22999,103092305.06
